In [1]:
import datetime
import glob
import IPython.display as ipd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import orca_params
import os
import pandas as pd
import random
import re
import shutil
import time
import urllib

from database_parser import extract_segment_features
from inference import create_network
from scipy.io import wavfile

Using TensorFlow backend.


A few files to play with...

```
7224  /KillerWhale/1997/97619014.wav  KillerWhale  16.558904
9020  /KillerWhale/1992/9230200E.wav  KillerWhale  15.351781
8484  /KillerWhale/1989/89405023.wav  KillerWhale  14.249452
8731  /KillerWhale/1992/9230201U.wav  KillerWhale  12.638082
9339  /KillerWhale/1960/6002603Y.wav  KillerWhale  12.319383
8701  /KillerWhale/1992/9230201H.wav  KillerWhale  12.095342
8939  /KillerWhale/1992/9230003C.wav  KillerWhale  11.764475
9116  /KillerWhale/1960/6002603O.wav  KillerWhale  11.374559
7211  /KillerWhale/1997/9750300N.wav  KillerWhale  11.367626
8716  /KillerWhale/1992/9230001V.wav  KillerWhale  11.304018


/Noise/OrcasoundLab/f8a882240e0f41bcb36f5be96eeb30b8.wav
/Noise/OrcasoundLab/f91ad2517ed54da9aefb885ffd8732ed.wav
/Noise/OrcasoundLab/f967c4ec1dd249a48d02d12e14a89a80.wav
/Noise/OrcasoundLab/f980dc3270864b579a9708e9765d577e.wav
/Noise/OrcasoundLab/fc26945d0ed54e44a7c95163cf79161b.wav
/Noise/OrcasoundLab/fe9fe361a0c24d87924f2646038315ed.wav
/Noise/OrcasoundLab/ff5fdad5e82347ff91df8f51b48f119b.wav

/Noise/BushPoint/f68b38de15ab4ad48ad5faede3d5a8a4.wav
/Noise/BushPoint/f6a48ccf967f457993b1089ca2040376.wav
/Noise/BushPoint/f7f89933353046e797330f2010ad1ec1.wav
/Noise/BushPoint/fabd4de7f993430c93cb39c0ec1d2054.wav
/Noise/BushPoint/fc86a77b07bc49dfa8d8388e72f32fa0.wav
/Noise/BushPoint/fc951c73c07343e0a299ce09bd128cd3.wav
/Noise/BushPoint/fceecee265dd47e4ba6555129bfd027d.wav
/Noise/BushPoint/fd283f965135450e991135beb46a72fd.wav
/Noise/BushPoint/fed9728629cc4da0bce9027b297a2276.wav
```

In [137]:
mammal_file = '/data/KillerWhale/1997/97619014.wav'
# noise_file = '/data/Noise/OrcasoundLab/a185bcb634de4901999189ac9ad2c21d.wav'
noise_file = '/data/Noise/BushPoint/f68b38de15ab4ad48ad5faede3d5a8a4.wav'
output_file = '/data/positive_input/mixed_%02d.wav'

## Mammal

In [4]:
#listen
ipd.display(ipd.Audio(mammal_file))

## Noise

In [9]:
ipd.display(ipd.Audio(noise_file))

## Mixed Result

We don't want the noise and the positive sample to be mixed at the same volume.

Tip from StackOverflow: https://stackoverflow.com/questions/14498539/how-to-overlay-downmix-two-audio-files-using-ffmpeg

```
ffmpeg -y \
  -i audio1.mp3 \
  -i audio2.mp3 \
  -filter_complex "[0:0]volume=0.09[a];[1:0]volume=1.8[b];[a][b]amix=inputs=2:duration=longest" \
  -c:a libmp3lame \
  output.mp3
```

This is a working example from our application:

```
ffmpeg -y \
  -i /data/Noise/BushPoint/f68b38de15ab4ad48ad5faede3d5a8a4.wav \
  -i /data/KillerWhale/1997/97619014.wav \
  -filter_complex "[0:0]volume=0.18[a];[1:0]volume=1.8[b];[a][b]amix=inputs=2:duration=first" \
  -t 10 \
  -f segment \
  -segment_time 1 \
  -c:a libmp3lame \
  /data/positive_input/mixed_%02d.wav
```

> NOTE: I remove `-c:a libmp3lame` which doesn't seem to be needed.

In [142]:

LIVE_FEED_SEGMENT_DURATION = 10  # orca_params.LIVE_FEED_ITERATION_SECONDS
LIVE_FEED_SEGMENT_LENGTH = 1  # orca_params.LIVE_FEED_SEGMENT_SECONDS

# These are the values that affect mixing volume
MAMMAL_VOLUME_LEVEL = 1.8
ORCASOUND_VOLUME_LEVEL = 0.036
BUSHPOINT_VOLUME_LEVEL = 0.18

if 'BushPoint' in noise_file:
    NOISE_VOLUME_LEVEL = BUSHPOINT_VOLUME_LEVEL
elif 'Orcasound' in noise_file:
    NOISE_VOLUME_LEVEL = ORCASOUND_VOLUME_LEVEL

In [152]:
filter = '[0:0]volume={}[a];[1:0]volume={}[b];[a][b]amix=inputs=2:duration=first'.format(NOISE_VOLUME_LEVEL, MAMMAL_VOLUME_LEVEL)
ffmpeg_cli = 'ffmpeg -y -i {} -i {} -filter_complex "{}" -t {} -f segment -segment_time {} {}' \
             .format(noise_file, mammal_file, filter, LIVE_FEED_SEGMENT_DURATION, LIVE_FEED_SEGMENT_LENGTH, output_file)

print(ffmpeg_cli)
os.system(ffmpeg_cli)

ffmpeg -y -i /data/Noise/BushPoint/f68b38de15ab4ad48ad5faede3d5a8a4.wav -i /data/KillerWhale/1997/97619014.wav -filter_complex "[0:0]volume=0.18[a];[1:0]volume=1.8[b];[a][b]amix=inputs=2:duration=first" -t 10 -f segment -segment_time 1 /data/positive_input/mixed_%02d.wav


0

In [153]:
# listen
for i in range(0, LIVE_FEED_SEGMENT_DURATION):
    sound_file = '/data/positive_input/mixed_{:02d}.wav'.format(i)
    print(sound_file)
    ipd.display(ipd.Audio(sound_file))

/data/positive_input/mixed_00.wav


/data/positive_input/mixed_01.wav


/data/positive_input/mixed_02.wav


/data/positive_input/mixed_03.wav


/data/positive_input/mixed_04.wav


/data/positive_input/mixed_05.wav


/data/positive_input/mixed_06.wav


/data/positive_input/mixed_07.wav


/data/positive_input/mixed_08.wav


/data/positive_input/mixed_09.wav
